In [1]:
import numpy as np
import requests

API_KEY = 'ei_7c14274991477b16092db6eb8032e84b9f73c9c9e665b5c8bec01333b86151f3'

X = (requests.get('https://studio.edgeimpulse.com/v1/api/18137/training/6/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/18137/training/6/y', headers={'x-api-key': API_KEY})).content



Store the data in a temporary file, and load it back through Numpy.


In [2]:
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]


In [3]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import sys, os, random
import tensorflow as tf
import datetime, os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import itertools

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "N", "o", "s" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

logdir = os.path.join("./logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
file_writer_cm = tf.summary.create_file_writer(logdir)

In [4]:
def set_batch_size(batch_size, train_dataset, validation_dataset):
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return train_dataset, validation_dataset


import io
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    threshold = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return figure

from tensorflow import keras
from sklearn import metrics

def log_confusion_matrix(epoch, logs):
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)

    cm = metrics.confusion_matrix(np.argmax(Y_test, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=classes_values)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)



In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

# model architecture
model = Sequential()
model.add(Reshape((int(input_length / 17), 17), input_shape=(input_length, )))
model.add(Conv1D(19, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.5))
model.add(Conv1D(35, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(classes, activation='softmax', name='y_pred'))

# this controls the learning rate
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset, validation_dataset = set_batch_size(BATCH_SIZE, train_dataset, validation_dataset)

# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

callbacks = [
   tf.keras.callbacks.TensorBoard(log_dir=logdir,
                histogram_freq=1,
               write_graph=True,
               write_images=True,
               update_freq='epoch',
               profile_batch=2,
               embeddings_freq=1)
]

#model.fit(train_dataset, epochs=150, validation_data=validation_dataset, verbose=2)
model.fit(train_dataset, epochs=180, validation_data=validation_dataset, verbose=2,callbacks=callbacks)

Epoch 1/180
92/92 - 0s - loss: 3.0040 - accuracy: 0.1141 - val_loss: 2.4022 - val_accuracy: 0.2595
Epoch 2/180
92/92 - 0s - loss: 2.3909 - accuracy: 0.2052 - val_loss: 2.1327 - val_accuracy: 0.4348
Epoch 3/180
92/92 - 0s - loss: 2.0137 - accuracy: 0.3173 - val_loss: 1.6509 - val_accuracy: 0.5761
Epoch 4/180
92/92 - 0s - loss: 1.6915 - accuracy: 0.4297 - val_loss: 1.2916 - val_accuracy: 0.7133
Epoch 5/180
92/92 - 0s - loss: 1.4124 - accuracy: 0.5394 - val_loss: 1.0187 - val_accuracy: 0.7785
Epoch 6/180
92/92 - 0s - loss: 1.2334 - accuracy: 0.6101 - val_loss: 0.8535 - val_accuracy: 0.8261
Epoch 7/180
92/92 - 0s - loss: 1.0842 - accuracy: 0.6467 - val_loss: 0.7131 - val_accuracy: 0.8601
Epoch 8/180
92/92 - 0s - loss: 0.9515 - accuracy: 0.6980 - val_loss: 0.6004 - val_accuracy: 0.8804
Epoch 9/180
92/92 - 0s - loss: 0.8749 - accuracy: 0.7259 - val_loss: 0.5189 - val_accuracy: 0.9022
Epoch 10/180
92/92 - 0s - loss: 0.8253 - accuracy: 0.7381 - val_loss: 0.4775 - val_accuracy: 0.9008
Epoch 11/

In [6]:
# Save the model to disk
# model.save('saved_model')

print(logdir)
print("TensorFlow version: ", tf.__version__)

./logs/20210221-005235
TensorFlow version:  2.3.0
